<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium # map rendering library

# import BeautifulSoup package for web scraping 
#!pip install bs4
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Parsing the HTML side

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)

## Get the PostalCode, Borough, and Neighborhood from the Wikipedia web

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []

# Get the rows
for tr in trs:
    row = tr.find_all('td')
    if row:
        rows.append(row)

#rows       

# Then for every row, we get the postalcode, borough and neighborhood
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

#lst

In [4]:
# define the dataframe columns
column_names = ['Postalcode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(lst,columns=column_names)

df.shape

(103, 3)

In [5]:
# Check M5A whether it has Regent Park, Harbourfront 
df.loc[df['Postalcode'] == 'M5A']

,Postalcode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Downloading the Data



In [6]:
!wget -q -O 'FSA_Longtitude_Latitude.csv' http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

print('Data downloaded!')

Data downloaded!


In [7]:
longtitude_latitude_df = pd.read_csv('FSA_Longtitude_Latitude.csv')

# Rename Postal code to Postalcode for merge
longtitude_latitude_df.rename(columns={'Postal Code': 'Postalcode'}, inplace=True)
longtitude_latitude_df.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_with_LongLat = pd.merge(df, longtitude_latitude_df, on="Postalcode", how='left')

In [9]:
# Check M5A whether it has the right Latitude and Longtitude
df_with_LongLat.loc[df['Postalcode'] == 'M5A']

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


## Create a map of Toronto with neighborhoods superimposed on top.

In [10]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.638093, -79.4665843.


In [11]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_with_LongLat['Latitude'], df_with_LongLat['Longitude'], df_with_LongLat['Borough'], df_with_LongLat['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Create a map of North York with neighborhoods superimposed on top.

In [12]:
northyork_data = df_with_LongLat[df_with_LongLat['Borough'] == 'North York'].reset_index(drop=True)

address = 'North York, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_with_LongLat['Latitude'], df_with_LongLat['Longitude'], df_with_LongLat['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


## Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'KILPJAZ5ZQ1E4PKG35SUOFPSG5G2YEIDCI3DMD1VAACW55NK' # your Foursquare ID
CLIENT_SECRET = 'YPGN5GXXVHJPOQIP05YQTXXS2PHOJELTO0ZLSKPDOI3Z4XD1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KILPJAZ5ZQ1E4PKG35SUOFPSG5G2YEIDCI3DMD1VAACW55NK
CLIENT_SECRET:YPGN5GXXVHJPOQIP05YQTXXS2PHOJELTO0ZLSKPDOI3Z4XD1


## Let's explore the first neighborhood in our dataframe.

In [14]:
northyork_data.loc[0, 'Neighborhood']

'Parkwoods'

In [15]:
# Get the neighborhood's latitude and longitude values.

neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [16]:
# Now, let's get the top 100 venues that are in Parkwoods within a radius of 1000 meters.

# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,Tim Hortons,Café,43.760668,-79.326368
3,Bruno's valu-mart,Grocery Store,43.746143,-79.324630
4,A&W,Fast Food Restaurant,43.760643,-79.326865


## Let's Explore Neighborhoods in North York

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Get the North York venues
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )

northyork_venues.head()

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant


In [18]:
# Number of venues for each neighborhood
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",29,29,29,29,29,29
Bayview Village,14,14,14,14,14,14
"Bedford Park, Lawrence Manor East",41,41,41,41,41,41
Don Mills,76,76,76,76,76,76
Downsview,65,65,65,65,65,65
"Fairview, Henry Farm, Oriole",44,44,44,44,44,44
Glencairn,33,33,33,33,33,33
Hillcrest Village,21,21,21,21,21,21
Humber Summit,10,10,10,10,10,10


## Analyze Each Neighborhood

In [19]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]


# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Fireworks Store,Fish & Chips Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Liquor Store,Locksmith,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,New American Restaurant,Noodle House,Office,Other Repair Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Lab,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Ski Chalet,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.068966,0.00000,0.034483,0.034483,0.000000,0.00,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.034483,0.034483,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0,0.034483,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.034483,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0

## Let's print each neighborhood along with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.07
1            Bank  0.07
2  Ice Cream Shop  0.03
3      Ski Chalet  0.03
4   Shopping Mall  0.03


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.14
1          Gas Station  0.14
2        Grocery Store  0.14
3                 Bank  0.14
4           Restaurant  0.07


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.07
2         Pizza Place  0.05
3      Sandwich Place  0.05
4                Park  0.05


----Don Mills----
                 venue  freq
0          Coffee Shop  0.08
1           Restaurant  0.08
2  Japanese Restaurant  0.07
3                  Gym  0.05
4                 Bank  0.04


----Downsview----
                   venue  freq
0            Coffee Shop  0.08
1  Vietnamese Restaurant  0.08
2                  Hotel  0.06
3          Grocery Store  0.06
4               Pha

## Create the new dataframe that display the top 10 venues for each neighborhood.

In [21]:
# Function to sort venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Deli / Bodega,Restaurant,Sandwich Place,Bridal Shop,Diner,Shopping Mall,Mediterranean Restaurant,Ski Area
1,Bayview Village,Japanese Restaurant,Grocery Store,Gas Station,Bank,Skating Rink,Café,Park,Trail,Chinese Restaurant,Restaurant
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Park,Sandwich Place,Bank,Pizza Place,Fast Food Restaurant,Grocery Store,Indian Restaurant,Restaurant
3,Don Mills,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Burger Joint,Bank,Supermarket,Beer Store,Café,Mobile Phone Shop
4,Downsview,Coffee Shop,Vietnamese Restaurant,Hotel,Grocery Store,Pharmacy,Gas Station,Pizza Place,Liquor Store,Turkish Restaurant,Fast Food Restaurant


## Cluster Neighborhoods

In [22]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge northyork_merged with manhattan_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# create map with the created clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Cluster 1

In [23]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Convenience Store,Shopping Mall,Bus Stop,Pharmacy,Grocery Store,Shop & Service,Cosmetics Shop,Discount Store,Fast Food Restaurant
1,North York,0,Coffee Shop,Hockey Arena,Grocery Store,Portuguese Restaurant,Men's Store,Playground,Sporting Goods Shop,Golf Course,Boxing Gym,Pizza Place
2,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Dessert Shop,Furniture / Home Store,Accessories Store
3,North York,0,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Burger Joint,Bank,Supermarket,Beer Store,Café,Mobile Phone Shop
4,North York,0,Grocery Store,Fast Food Restaurant,Coffee Shop,Gas Station,Italian Restaurant,Pizza Place,Playground,Park,Pub,Convenience Store
5,North York,0,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Burger Joint,Bank,Supermarket,Beer Store,Café,Mobile Phone Shop
6,North York,0,Pharmacy,Park,Coffee Shop,Housing Development,Bakery,Sandwich Place,Restaurant,Residential Building (Apartment / Condo),Korean Restaurant,Recreation Center
7,North York,0,Bank,Coffee Shop,Deli / Bodega,Restaurant,Sandwich Place,Bridal Shop,Diner,Shopping Mall,Mediterranean Restaurant,Ski Area
8,North York,0,Clothing Store,Coffee Shop,Sandwich Place,Japanese Restaurant,Bank,Bakery,Restaurant,Juice Bar,Chocolate Shop,Caribbean Restaurant
9,North York,0,Coffee Shop,Furniture / Home Store,Pizza Place,Restaurant,Fast Food Restaurant,Bar,Sandwich Place,Metro Station,Middle Eastern Restaurant,Road


## Examine Cluster 2

In [24]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,1,Park,Pool,Discount Store,Fast Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run


## Examine Cluster 3

In [25]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2,Convenience Store,Storage Facility,Intersection,Park,Auto Workshop,Discount Store,Gas Station,Bakery,Golf Course,Fast Food Restaurant


## Examine Cluster 4

In [26]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,3,Electronics Store,Pizza Place,Pharmacy,Park,Shopping Mall,Italian Restaurant,Medical Center,Bakery,Bank,Dumpling Restaurant


## Examine Cluster 5

In [27]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,4,Pharmacy,Pizza Place,Convenience Store,Bus Line,Coffee Shop,Park,Grocery Store,Eastern European Restaurant,Butcher,Bakery
